In [4]:
import pandas as pd
import numpy as np

# 1. Read Data

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [8]:
train.shape

(3000, 23)

In [9]:
test.shape

(4398, 22)

# 2.Clean Data

There seems to be way more test cases than training cases ideally we'd want it the other way around. Lets clean up these data sets a bit.

In [15]:
train.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


The following columns can be dropped from our two datasets: belongs_to_collection, homepage, imdb_id, original_title, overview, poster_path, status, tagline.

In [18]:
drop_cols = ['belongs_to_collection','homepage','imdb_id','original_title','overview','poster_path','status','tagline']

In [19]:
for col in drop_cols:
    train.drop(col, axis=1, inplace = True)
    test.drop(col, axis=1, inplace = True)

In [21]:
train.shape

(3000, 15)

In [22]:
test.shape

(4398, 14)

In [24]:
train.columns

Index(['id', 'budget', 'genres', 'original_language', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'runtime', 'spoken_languages', 'title', 'Keywords', 'cast', 'crew',
       'revenue'],
      dtype='object')

Now lets clean up the columns individually

## 2.1 genres
The genres column contains a dictionary for each entry. The dictionary contains each genre the film is apart of, some films are apart of more than one genre. Lets do something about the the null genres for each entry and see if we can add them ourselves

In [28]:
test.genres.isnull().sum()

0

In [29]:
train.genres.isnull().sum()

7

In [67]:
genres = train.genres

In [73]:
genres_null = train.genres.isnull()

In [74]:
x = 0
null_indx = []
for value in genres_null:
    if value == True:
        null_indx.append(x)
    x = x + 1

In [78]:
null_indx

[470, 1622, 1814, 1819, 2423, 2686, 2900]

In [77]:
for index in null_indx:
    print(train.loc[index,:])

id                                                                    471
budget                                                            2000000
genres                                                                NaN
original_language                                                      en
popularity                                                       0.079856
production_companies                                                  NaN
production_countries                                                  NaN
release_date                                                      9/12/03
runtime                                                               120
spoken_languages                                                      NaN
title                     The Book of Mormon Movie, Volume 1: The Journey
Keywords                                                              NaN
cast                    [{'cast_id': 1, 'character': 'Sam', 'credit_id...
crew                                  

Now that we have all the movies with null genres, we can fill them out ourselves